In [6]:
import torch
import numpy as np
import sys

sys.path.append('../human_body_prior/tools/')
from omni_tools import copy2cpu as c2c
from os import path as osp

both57m_dir = '../data/both57m/'
comp_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(comp_device)

sys.path.append('../human_body_prior/body_model/')
from body_model import BodyModel

gender = "male"

bm_fname = '../smplh_models/{}/model.npz'.format(gender)
bm = BodyModel(bm_fname=bm_fname).to(comp_device)
faces = c2c(bm.f)

cuda


In [7]:
import trimesh
sys.path.append('../body_visualizer')
from tools.vis_tools import colors
from mesh.mesh_viewer import MeshViewer
from mesh.sphere import points_to_spheres
from tools.vis_tools import show_image

imw, imh=1600, 1600
mv = MeshViewer(width=imw, height=imh, use_offscreen=True)

data_number = 500

both57m_npz = osp.join(both57m_dir, str(data_number), 'data.npz')
bdata = np.load(both57m_npz)
len = bdata['seq_len']

motion_parms = {
    'root_orient': torch.Tensor(bdata['root_orient']).to(comp_device),
    'pose_body': torch.Tensor(bdata['body_pose'].reshape(len,-1)).to(comp_device),
    'pose_hand': torch.Tensor(bdata['hand_pose'].reshape(len,-1)).to(comp_device),
    'trans': torch.Tensor(bdata['trans']).to(comp_device)
}

body_pose = bm(**{k:v for k,v in motion_parms.items() if k in ['pose_body', 'pose_hand', 'root_orient']})
joints = np.zeros((len, 52, 3))

for i in range(len):
    array = c2c(body_pose.Jtr[i])
    array = array[:,[2,0,1]]
    joints[i] = array

print(type(joints), joints.shape)
np.save('joints.npy', joints)

del bm
del mv

<class 'numpy.ndarray'> (150, 52, 3)


In [8]:
SMPLH_JOINT_NAMES = ['pelvis','left_hip','right_hip','spine1','left_knee',
    'right_knee','spine2','left_ankle','right_ankle','spine3','left_foot',
    'right_foot','neck','left_collar','right_collar','head',
    'left_shoulder','right_shoulder','left_elbow','right_elbow','left_wrist','right_wrist',
    'left_index1', 'left_index2','left_index3',
    'left_middle1','left_middle2','left_middle3',
    'left_pinky1','left_pinky2','left_pinky3',
    'left_ring1','left_ring2','left_ring3',
    'left_thumb1','left_thumb2','left_thumb3',
    'right_index1','right_index2','right_index3',
    'right_middle1','right_middle2','right_middle3',
    'right_pinky1','right_pinky2','right_pinky3',
    'right_ring1','right_ring2','right_ring3',
    'right_thumb1','right_thumb2','right_thumb3']

In [9]:
import mujoco
import numpy as np
import matplotlib.pyplot as plt
from mujoco_parser import MuJoCoParserClass
from util import rpy2R,r2quat,get_uv_dict_nc,get_p_target_nc
np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("MuJoCo version:[%s]"%(mujoco.__version__))

xml_path = '../asset/smplh_rig/scene_smplh_rig.xml'
env = MuJoCoParserClass(name='Common-Rig',rel_xml_path=xml_path,VERBOSE=True)
# Modify the color of body exluding 'world'
for body_name in env.body_names:
    if body_name in ['world']: continue
    for geom_idx in env.get_geom_idxs_from_body_name(body_name=body_name):
        env.model.geom(geom_idx).rgba = [0.3,0.3,0.5,0.5]

MuJoCo version:[3.1.2]
dt:[0.0020] HZ:[500]
n_dof (=nv):[89]
n_geom:[75]
geom_names:['floor', 'base', 'base2lpelvis', 'base2rpelvis', 'base2spine1', 'spine12spine2', 'spine22spine3', 'spine32neck', 'spine32lcollar', 'spine2rcollar', 'neck2head', 'head', 'nose', 'lcollar2lshoulder', 'lshoulder2lelbow', 'lelbow2lwrist', 'lwrist2lindex0', 'lwrist2lmiddle0', 'lwrist2lpinky0', 'lwrist2lring0', 'lwrist2lthumb1', 'lindex0-lindex1', 'lindex1-lindex2', 'lindex2-lindex3', 'lindex3-lindextip', 'lmiddle0-lmiddle1', 'lmiddle1-lmiddle2', 'lmiddle2-lmiddle3', 'lmiddl3-lmiddletip', 'lpinky0-lpinky1', 'lpinky1-lpinky2', 'lpinky2-lpinky3', 'lpinky3-lpinkytip', 'lring0-lring1', 'lring1-lring2', 'lring2-lring3', 'lring3-lringtip', 'lthumb1-lthumb2', 'lthumb2-lthumb3', 'lthumb3-lthumbtip', 'rcollar2rshoulder', 'rshoulder2relbow', 'relbow2rwrist', 'rwrist2rindex0', 'rwrist2rmiddle0', 'rwrist2rpinky0', 'rwrist2rring0', 'rwrist2rthumb1', 'rindex0-rindex1', 'rindex1-rindex2', 'rindex2-rindex3', 'rindex3-rindex

In [ ]:
joints = np.load("joints.npy")
print(joints.shape)

# # Initialize MuJoCo viewer
# env.init_viewer(viewer_title='Common-Rig',viewer_width=1200,viewer_height=800,
#                 viewer_hide_menus=True)
# env.update_viewer(azimuth=180,distance=2.6,elevation=-25,lookat=[0.0,0.0,0.83])
# env.reset()

# while env.is_viewer_alive():
#     env.forward(q=env.data.qpos.copy(),INCREASE_TICK=True)

#     if env.loop_every(tick_every=10):
#         for i in range(52):
#             env.plot_T(p=joints[0,i,:],R=np.eye(3),
#                         PLOT_AXIS=True,axis_len=0.1,axis_width=0.002)

#         env.render()

# env.close_viewer()